In [1]:
total_size_hc = total_size_pth = 100
pre_mean = 20
pre_sd = 1.5
pre_min = 15
pre_max = 28
post_min = 0
post_max = 28
natural_variability_mean = 0
natural_variability_sd =0.5

natural_improvement_min_pct = 0.50 
natural_improvement_max_pct = 1
natural_improvers_pct =0.25 # .25

#this isnt' really hc vs pth, this would be all pth treated vs untreated but here ... 
treated_pct = 1
treatment_improvement_mean = .2
treatment_improvement_sd = .15
#treatment_improvement_min_pct = 0.0#250 
#treatment_improvement_max_pct = 1
n_replications<-1000
#should it be like a zero inflated? chance of response and then response? 
model_parameters<-c(total_size_hc,  pre_mean, pre_sd,pre_min,pre_max, post_min,post_max, natural_variability_mean, natural_variability_sd,
  natural_improvement_min_pct, natural_improvement_max_pct, natural_improvers_pct, treated_pct, treatment_improvement_mean,
  treatment_improvement_sd,n_replications)
  
simulation<-function(){
    d1 = data.frame(cbind("group" = c(rep("hc",total_size_hc),rep("pth",total_size_pth)), "pre" =base_frequency_fun(total_size_pth+total_size_hc)))
    names(d1)<-c("group","pre")
    d1$pre<-as.numeric(d1$pre)
    d1['natural_improver']<-0
    d1[sample(seq(1,dim(d1)[1]),dim(d1)[1]*natural_improvers_pct),'natural_improver']<-1
    d1['intervention']<-0
    d1[sample(row.names(subset(d1, group == "pth")),treated_pct *total_size_pth),'intervention']<-1
    
    d1["post"]<-apply(d1, 1, natural_variability_fun, "pre")
    
    if(sum(d1$intervention) >0){
      d1[d1$intervention ==1,"post"]<-apply(d1[d1$intervention ==1,],1,intervention_improvement_fun, "post")
    }else{
      print('no intervention')
    }
    if(sum(d1$natural_improver) >0){
      d1[d1$natural_improver == 1,"post"]<-apply(d1[d1$natural_improver == 1 ,], 1, natural_improvement_fun, "post")
    }
    if(sum(1- d1$natural_improver) >0){
      d1[d1$natural_improver == 0,"post"]<-apply(d1[d1$natural_improver == 0 ,], 1, natural_variability_fun, "post")
    }
    d1['change']<-d1$pre - d1$post
    d1['change_pct']<-d1$change/d1$pre
    c(as.numeric(unlist(aggregate(change ~ group, d1, mean))[3:4]),
      t.test( subset(d1, group == "pth")$change, subset(d1, group == "hc")$change, alternative = "greater")$p.value,
      model_parameters)
}



base_frequency_fun<-function(sz){
  vls<-do.call(rbind,lapply(seq(1,sz,1), FUN = function(normally_distributed_value = NA){
  while (is.na(normally_distributed_value) || normally_distributed_value < pre_min || normally_distributed_value > pre_max) {
    normally_distributed_value <- rnorm(1, mean = pre_mean, sd = pre_sd)
  }
    round(normally_distributed_value,0)
    }))
  vls
}


natural_improvement_fun<-function(x,var){
  #gives a % improvement
  xx = as.numeric(x[[var]])
  max(0,round(xx - xx*runif(1,natural_improvement_min_pct, natural_improvement_max_pct),0))
}



natural_variability_fun<-function(x, var){
  #gives a number 
  xx = as.numeric(x[[var]])
  min(max(0,round(xx + rnorm(1,natural_variability_mean, natural_variability_sd)),0),28)
}

#should intervention make natural decrease even more? 
intervention_improvement_fun<-function(x,var){
  #gives a % improvement
  xx = as.numeric(x[[var]])
  max(0,round(xx - xx*rnorm(1,treatment_improvement_mean, treatment_improvement_sd),0))
}


In [2]:
res<-as.data.frame(t(replicate(n_replications,simulation())))
names(res)<-c("change_ctrl","change_intervention","p.value", "total_size_hc",  "pre_mean", "pre_sd","pre_min","pre_max", "post_min",
              "post_max", "natural_variability_mean", "natural_variability_sd",
              "natural_improvement_min_pct", "natural_improvement_max_pct", "natural_improvers_pct", "treated_pct", "treatment_improvement_mean",
              "treatment_improvement_sd","n_replications" )
res['treatment_effect']<-res$change_intervention - res$change_ctrl

mean(res$treatment_effect)
mean(res$treatment_effect)  - 2*sd(res$treatment_effect)
table(res$p.value<=0.05)
#write.csv(res, paste("C:/Users/m090652/OneDrive - Mayo Clinic/H_Mig/Fisher Dodick AI/simulation pth id/pct_natural_",natural_improvers_pct,".csv",sep = ""), row.names = FALSE)
#


[1] 0.94731

[1] 0.3318871


FALSE  TRUE 
   97   903 